<a href="https://colab.research.google.com/github/harshit433/ANN-from-scratch/blob/main/ANN_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import keras
import time
from sklearn.metrics import r2_score, log_loss, accuracy_score
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
import torch
import math


class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.w1 = torch.nn.Parameter(torch.randn((500,1000)))
        self.w2 = torch.nn.Parameter(torch.randn((1,500)))
        self.bs1 = torch.nn.Parameter(torch.randn((500,1)))
        self.bs2 = torch.nn.Parameter(torch.randn((1,1)))
        torch.nn.init.xavier_uniform_(self.w1)
        torch.nn.init.xavier_uniform_(self.w2)
        torch.nn.init.zeros_(self.bs1)
        torch.nn.init.zeros_(self.bs2)


    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        z1 = torch.nn.ReLU()(torch.mm(self.w1, x.T) + self.bs1)
        z2 = torch.nn.ReLU()(torch.mm(self.w2, z1)  + self.bs2)
        return z2


    def fit(self,train_dataloader,test_dataloader,learning_rate,epochs):
      # Construct our model by instantiating the class defined above

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters (defined
# with torch.nn.Parameter) which are members of the model.
      criterion = torch.nn.MSELoss()
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
      loss_list = []
      accuracy_list = []
      # Training loop
      for epoch in range(epochs):

      # Forward pass: Compute predicted y by passing x to the model
        for X,y in train_dataloader:
          y_pred = model(X)

      # Compute and print loss
          loss = criterion(y_pred.T, y)

      # Zero gradients, perform a backward pass, and update the weights.
          optimizer.zero_grad()
          loss.backward(retain_graph = True)
          optimizer.step()

          accuracy = r2_score(y, model(X).detach().numpy().reshape(64,1))

          loss_list.append(loss.item())
          accuracy_list.append(accuracy)
          print(epoch,'loss: ', loss_list[-1] , 'accuracy: ', accuracy_list[-1])



In [ ]:
# Convert data to torch tensors
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.len


In [ ]:
df = pd.read_csv('/content/insurance.csv')

In [ ]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [ ]:
df['sex'] = df['sex'].map({'female':1,'male':2})
df['smoker'] = df['smoker'].map({'yes':2,'no':1})
df['region'] = df['region'].map({'southwest':1,'southeast':2,'northwest':2,'northeast':4})

In [ ]:
X = df.drop(['charges'], axis=1)
y = df['charges']

In [ ]:
x = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled)

In [ ]:
y = y.values.reshape(-1,1) #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
y_scaled = min_max_scaler.fit_transform(y)
y = pd.DataFrame(y_scaled)

In [ ]:
X_train  = X[:900]
X_val = X[900:1100]
Y_train = y[:900]
Y_val = y[900:1100]
X_test = X[1100:]
Y_test = y[1100:]

In [ ]:
X_train = X_train.to_numpy(dtype = np.float32)
Y_train = Y_train.to_numpy(dtype = np.float32)
X_test = X_test.to_numpy(dtype = np.float32)
Y_test = Y_test.to_numpy(dtype = np.float32)
X_val = X_val.to_numpy(dtype = np.float32)
Y_val = Y_val.to_numpy(dtype = np.float32)

In [ ]:
X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train)
X_test = torch.from_numpy(X_test)
Y_test = torch.from_numpy(Y_test)

X_val = torch.from_numpy(X_val)
Y_val = torch.from_numpy(Y_val)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)
print(X_val.shape, Y_val.shape)

torch.Size([900, 6]) torch.Size([900, 1])
torch.Size([238, 6]) torch.Size([238, 1])
torch.Size([200, 6]) torch.Size([200, 1])


In [ ]:
inp = np.random.randn(1000,100000).T
out = np.random.uniform(low = 1, high = 10, size = (100000,1))

X_train  = inp[:74944]
X_test = inp[74944:]
Y_train = out[:74944]
Y_test = out[74944:]

batch_size = 64

# Instantiate training and test data
train_data = Data(X_train, Y_train)
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

test_data = Data(X_test, Y_test)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

In [ ]:
model = Polynomial3()

In [ ]:
model.fit(train_dataloader,test_dataloader,learning_rate=0.001,epochs = 10)

Streaming output truncated to the last 5000 lines.
5 loss:  1.7196677923202515 accuracy:  0.8187130083234365
5 loss:  1.851212978363037 accuracy:  0.7964017238042667
5 loss:  1.8538674116134644 accuracy:  0.7993507200039447
5 loss:  1.8028231859207153 accuracy:  0.8189900722078178
5 loss:  1.670983076095581 accuracy:  0.8391293309827057
5 loss:  1.52149498462677 accuracy:  0.8793765144932761
5 loss:  1.5027776956558228 accuracy:  0.8182652370020161
5 loss:  1.6224390268325806 accuracy:  0.8345928449049141
5 loss:  1.916886568069458 accuracy:  0.8053456284027889
5 loss:  1.521904468536377 accuracy:  0.8628440087704823
5 loss:  1.7986418008804321 accuracy:  0.8191908274677144
5 loss:  2.484008550643921 accuracy:  0.7718823913287216
5 loss:  2.072204828262329 accuracy:  0.7985631856475579
5 loss:  2.2686519622802734 accuracy:  0.7593521493169995
5 loss:  2.0328173637390137 accuracy:  0.7726410340785481
5 loss:  1.5082509517669678 accuracy:  0.8392000709199039
5 loss:  1.8756905794143677 a

In [ ]:
kerasmodel = keras.Sequential([
    keras.layers.Dense(500,input_dim= 1000, activation='relu'),
    keras.layers.Dense(1, activation='relu'),
    ])

In [ ]:
kerasmodel.compile(optimizer='Adam', loss='mean_squared_error',metrics = [keras.metrics.R2Score()])
start_time = time.time()
kerasmodel.fit(X_train, Y_train, epochs=10, batch_size=32)
print("Using New Approach --- %s seconds ---" % (time.time() - start_time))

Epoch 1/10
2343/2343 [==============================] - 21s 9ms/step - loss: 7.7883 - r2_score: -0.1544
Epoch 2/10
2343/2343 [==============================] - 20s 9ms/step - loss: 6.4621 - r2_score: 0.0422
Epoch 3/10
2343/2343 [==============================] - 22s 9ms/step - loss: 5.8039 - r2_score: 0.1397
Epoch 4/10
2343/2343 [==============================] - 20s 9ms/step - loss: 5.1675 - r2_score: 0.2341
Epoch 5/10
2343/2343 [==============================] - 20s 9ms/step - loss: 4.3897 - r2_score: 0.3494
Epoch 6/10
2343/2343 [==============================] - 22s 9ms/step - loss: 3.1170 - r2_score: 0.5380
Epoch 7/10
2343/2343 [==============================] - 21s 9ms/step - loss: 1.9793 - r2_score: 0.7066
Epoch 8/10
2343/2343 [==============================] - 21s 9ms/step - loss: 1.4210 - r2_score: 0.7894
Epoch 9/10
2343/2343 [==============================] - 20s 9ms/step - loss: 1.1780 - r2_score: 0.8254
Epoch 10/10
2343/2343 [==============================] - 20s 8ms/step - 